In [ ]:
!rm -rf TwoTimePad
!git clone --depth 1 https://github.com/matthiasgoergens/TwoTimePad.git
!rm -rf data *.h5
!cp -r TwoTimePad/data TwoTimePad/*.h5 ./
!pip install -q --no-deps tensorflow-addons

In [2]:
%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  print(f'Found device at: {device_name}')
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [3]:

device_name = tf.test.gpu_device_name()

if device_name != '/device:GPU:0':
  print(
      '\n\nThis error most likely means that this notebook is not '
      'configured to use a GPU.  Change this in Notebook Settings via the '
      'command palette (cmd/ctrl-shift-P) or the Edit menu.\n\n')
  raise SystemError('GPU device not found')

with tf.device('/device:GPU:0'):
  import pathlib
  import re

  import tensorflow
  import tensorflow as tf
  from tensorflow import keras


  import pathlib
  import numpy as np
  import random
  from pprint import pprint

  # Train with longer later.
  # n = 25
  # n = 50

  np.set_printoptions(precision=4)


  # Idea:
  # Load corpus, and clean, convert into numbers
  # Repeatedly:
  #   create a batch of data
  #   train
  #   Optional: show current results.

  alpha =    " ABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789.?,-:;'()".lower()
  alphaRE = alpha.replace("-","\\-")

  assert len(alpha) == 46

  def load():
    text = ' '.join(f.open('r').read() for f in pathlib.Path('data').glob('*.txt')).lower()
    text = re.sub('\s+', ' ', text)
    text = re.sub(f'[^{alphaRE}]', '', text)
    return text

  def add(clear, key):
    return [(a+b) % len(alpha) for a, b in zip(clear, key)]

  def sub(cipher, key):
    return [(a-b) % len(alpha) for a, b in zip(cipher, key)]

  def clean(text):
    t = {c: i for i, c in enumerate(alpha)}
    return [t[c] for c in text]

  def toChar(numbers):
    return ''.join(alpha[i] for i in numbers)

  # toChar
  # (5, 125, 46)
  def toChars(tensor):
    (linesNums, charNum, alphaNum) = tensor.shape
    output = []
    # TODO: use gather https://www.tensorflow.org/api_docs/python/tf/gather?version=stable
    assert alphaNum == len(alpha)
    for lineNum in range(linesNums):
      chars = []
      for cN in range(charNum):
        (_, char) = max([(tensor[lineNum, cN, alphaN], alphaN) for alphaN in range(alphaNum)])
        chars.append(char)
      output.append(toChar(chars))
    return output

  def toChars_labels(labels):
    (linesNums, charNum) = labels.shape
    output = []
    for lineNum in range(linesNums):
      chars = []
      for cN in range(charNum):
        chars.append(labels[lineNum, cN])
      output.append(toChar(chars))
    return output


  def prepare(clear, key):
    assert len(clear) == len(key), (clear, key)

    # depth = len(alpha)

    # label = clear[len(clear)//2:len(clear)//2+1]
    return sub(clear, key)



  def sample(text, l):
    start = random.randrange(len(text) - l)
    return text[start:start+l]

  # def slicer(n):
  #   mid = round(n/2)
  #   return slice(mid, mid + 1)

  def slicer(n):
    return slice(20, -20)
    return slice(round(n/3), round (2 * n / 3) + 1)

  def samples(text, batch_size, l):
    ciphersA = []
    ciphersB = []
    keys = []
    labelsA = []
    labelsB = []
    for _ in range(batch_size):
      clearA = sample(text, l)
      clearB = sample(text, l)

      cipherA = prepare(clearA, clearB)

      ciphersA.append(cipherA)
      labelsA.append(clearA[slicer(l)])
      labelsB.append(clearB[slicer(l)])

    one_hot_ciphersA = tf.convert_to_tensor(ciphersA)

    return ([one_hot_ciphersA], [tf.convert_to_tensor(labelsA)])

  # relu = lambda: keras.activations.relu # (alpha=0.1)
  relu = lambda: keras.layers.LeakyReLU(alpha=0.1)
  # keras.layers.Embedding

  from tensorflow.keras.layers import Layer, Concatenate, Embedding, Input, Dense, Dropout, Softmax, GlobalMaxPooling1D, MaxPooling1D, Conv1D, Flatten, concatenate, Bidirectional, LSTM, SimpleRNN, Add
  from tensorflow_addons.optimizers import CyclicalLearningRate
  from tensorflow.keras.models import Sequential, Model
  import tensorflow_addons as tfa

  Maxout = tfa.layers.Maxout
  def make_model(n):
    inputA = Input(shape=(n,), dtype='int32', name="cipherA")

    emb = Embedding(output_dim=len(alpha), input_dim=len(alpha), name="my_embedding")
    embedded = emb(inputA)

    # relud = lambda base: relu()(Dense()(base))
    # Idea is to bubble up better and better guesses for what letters we have locally.
    
    # I don't think the dropout here is a great idea?
    # Perhaps have the same dropout for each layer instead?

    dropout = tf.keras.layers.SpatialDropout1D(0.5)
    conv1 = lambda base: (Conv1D(
        filters=46*2*2 * 5, kernel_size=5,
        padding='same', strides=1)(
          base
        ))

    pseudo_maxout = lambda l: tfa.layers.Maxout(46*2*2)(l)
    pseudo_maxout = lambda l: relu()(Conv1D(
        filters=46*2*2, kernel_size=1,
        padding='same', strides=1)(relu()(l)))

    conv = lambda base: dropout(Add()([base, conv1(pseudo_maxout(base))]))
    # Not sure whether the eta abstraction is necessary?
    lstm_ed = (
      (conv(
      (conv(
      (conv(
      (conv(
      (conv(
      (conv(
      (conv(
      (conv(
      conv1(embedded)
    )))))))))))))))))

    totes_clear = (
      (Conv1D(
        filters=len(alpha), kernel_size=5,
        padding='same', strides=1)(
      (lstm_ed)
    )))[:, slicer(n), :]
    model = Model([inputA], [Layer(name="clear")(totes_clear)])

    model.compile(
      optimizer=tf.keras.optimizers.Adam(),
      loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
      metrics=['accuracy'])
    return model

  import sys

  # TODO: implement generator or read from file?
  # Or use dataset and augment?
  training_size = 2 * (2**10) ** 2
  test_size = 10**3
  batch_size = 512

update_freq = 200

from datetime import datetime
name = '2021-June'
logdir = f"/content/drive/My Drive/twotimepad/logs/fit2/{datetime.now().strftime('%Y%m%d-%H%M%S')}_{name}"
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=logdir, profile_batch=0, update_freq='batch')
# update_freq -> set to more often.

from keras.callbacks import *
weights_name = f'/content/drive/My Drive/twotimepad/{name}-checkpoint.tf'
# checkpoint = ModelCheckpoint(weights_name, monitor='loss', verbose=1, save_best_only=True, save_freq=update_freq)
checkpoint = ModelCheckpoint(weights_name, verbose=1, save_freq=update_freq)

from tensorflow.keras.callbacks.experimental import BackupAndRestore
# from pathlib import Path
# Path(f'/content/drive/My Drive/twotimepad/{name}/foo').touch()
backup = BackupAndRestore(f'/content/drive/My Drive/twotimepad/{name}-backup_restore/')

callbacks_list = [checkpoint, tensorboard_callback, backup]


In [4]:
with tf.device('/device:GPU:0'):
  from google.colab import drive
  drive.mount('/content/drive')

  import sys
  


Mounted at /content/drive


In [5]:
start_l = 100
max_l = 100

In [ ]:
import itertools as it
first = True

# epochs_batch = 10
l = start_l
model = make_model(l)
model.summary()
text = clean(load())
try:
  model.load_weights(weights_name)
  print("Loaded weights.")
except Exception as e:
  print(f"Got: {e}")
  print("Failed to load weights.  Starting from scratch.")

with tf.device('/device:GPU:0'):
  for epoch in it.count():
    # l = min(max_l, l)

    print("Predict:")
    predict_size = 5
    print(f"samples(text, predict_size, l): {predict_size}, {l})")
    (inputs_p, labels_p) = samples(text, predict_size, l)
    [ciphersA_p] = labels_p
    predictA_p = model.predict(inputs_p)
    pprint(list(zip(

        toChars_labels(ciphersA_p),
        toChars(predictA_p),

        )),
      width=l+10)
    print("Training:")
    (inputs, [labels]) = samples(text, training_size, l)

    model.fit(x=inputs, y=labels,
              batch_size=batch_size,
              # validation_data=(ciphers_t, [labels_t, keys_t]),
              callbacks=callbacks_list,
              # epochs=epochs_batch * (epoch+1), initial_epoch=epochs_batch  * epoch
              validation_split=0.01,
              epochs=100,
              )


Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
cipherA (InputLayer)            [(None, 100)]        0                                            
__________________________________________________________________________________________________
my_embedding (Embedding)        (None, 100, 46)      2116        cipherA[0][0]                    
__________________________________________________________________________________________________
conv1d (Conv1D)                 (None, 100, 920)     212520      my_embedding[0][0]               
__________________________________________________________________________________________________
leaky_re_lu_1 (LeakyReLU)       (None, 100, 920)     0           conv1d[0][0]                     
______________________________________________________________________________________________